In [29]:
import pandas as pd
import requests
import pprint

response = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados')
dados = response.json()
df_deputados = pd.DataFrame(dados['dados'])
df_deputados_id = df_deputados['id']
df_deputados_nome = df_deputados['nome']
display(df_deputados_nome)

0       Abilio Brunini
1       Acácio Favacho
2          Adail Filho
3      Adilson Barroso
4         Adolfo Viana
            ...       
507           Zé Silva
508          Zé Trovão
509           Zé Vitor
510        Zeca Dirceu
511    Zezinho Barbary
Name: nome, Length: 512, dtype: object

In [92]:
contador = 0
df = pd.DataFrame()
for requisicao in range(0,512):
    print('Carregando as despesas do deputado {} [...]'.format(df_deputados_nome[contador]))
    response_despesas = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados/{}/despesas?ano=2020&ano=2021&ano=2022&ano=2023&ordem=ASC&ordenarPor=ano'.format(df_deputados_id[contador]))
    dados = response_despesas.json()
    df_despesas = pd.DataFrame(dados['dados'])
    df_despesas['nome_deputado'] = df_deputados_nome[contador]
    df = pd.concat([df, df_despesas])
    print('Despesas do deputado {} adicionado aos dados para análise!'.format(df_deputados_nome[contador]))
    contador = contador + 1

Carregando as despesas do deputado Abilio Brunini [...]
Despesas do deputado Abilio Brunini adicionado aos dados para análise!
Carregando as despesas do deputado Acácio Favacho [...]
Despesas do deputado Acácio Favacho adicionado aos dados para análise!
Carregando as despesas do deputado Adail Filho [...]
Despesas do deputado Adail Filho adicionado aos dados para análise!
Carregando as despesas do deputado Adilson Barroso [...]
Despesas do deputado Adilson Barroso adicionado aos dados para análise!
Carregando as despesas do deputado Adolfo Viana [...]
Despesas do deputado Adolfo Viana adicionado aos dados para análise!
Carregando as despesas do deputado Adriana Ventura [...]
Despesas do deputado Adriana Ventura adicionado aos dados para análise!
Carregando as despesas do deputado Adriano do Baldy [...]
Despesas do deputado Adriano do Baldy adicionado aos dados para análise!
Carregando as despesas do deputado Aécio Neves [...]
Despesas do deputado Aécio Neves adicionado aos dados para a

In [93]:
display(df)

,nome_deputado,ano,mes,tipoDespesa,codDocumento,tipoDocumento,codTipoDocumento,dataDocumento,numDocumento,valorDocumento,urlDocumento,nomeFornecedor,cnpjCpfFornecedor,valorLiquido,valorGlosa,numRessarcimento,codLote,parcela
0,Acácio Favacho,2020.0,2.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7018213.0,Nota Fiscal Eletrônica,4.0,2020-02-18,1790,145.00,http://www.camara.leg.br/cota-parlamentar/nota...,BEL PONTO BORDADOS E CONFECCOES LTDA,00849026000109,145.00,0.0,,1678271.0,0.0
1,Acácio Favacho,2020.0,6.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7065252.0,Nota Fiscal Eletrônica,4.0,2020-06-16,545,229.00,http://www.camara.leg.br/cota-parlamentar/nota...,COMERCIAL DE SERVICOS UNIAO LTDA ME,05807200000174,229.00,0.0,,1702259.0,0.0
2,Acácio Favacho,2020.0,2.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7031733.0,Nota Fiscal,0.0,2020-03-03,417246,268.40,https://www.camara.leg.br/cota-parlamentar/doc...,EDVALDO FRANCISCO DE OLIVEIRA,32373812000179,268.40,0.0,,1684292.0,0.0
3,Acácio Favacho,2020.0,1.0,COMBUSTÍVEIS E LUBRIFICANTES.,6993481.0,Nota Fiscal Eletrônica,4.0,2020-01-06,479,3600.00,http://www.camara.leg.br/cota-parlamentar/nota...,A C CAMBRAIA LTDA,23068984000571,3600.00,0.0,,1666719.0,0.0
4,Acácio Favacho,2020.0,8.0,COMBUSTÍVEIS E LUBRIFICANTES.,7090596.0,Nota Fiscal Eletrônica,4.0,2020-08-03,128126,200.02,http://www.camara.leg.br/cota-parlamentar/nota...,AUTO POSTO 306 NORTE LTDA,22398818000147,200.02,0.0,,1715262.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,Zeca Dirceu,2020.0,7.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7092392.0,Recibos/Outros,1.0,2020-07-31,sn,1300.00,https://www.camara.leg.br/cota-parlamentar/doc...,Antonio Peguim,04312880972,1300.00,0.0,,1716367.0,0.0
11,Zeca Dirceu,2020.0,8.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7092393.0,Recibos/Outros,1.0,2020-08-30,sn,1300.00,https://www.camara.leg.br/cota-parlamentar/doc...,Antonio Peguim,04312880972,1300.00,0.0,,1716365.0,0.0
12,Zeca Dirceu,2020.0,1.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7019946.0,Nota Fiscal,0.0,2020-01-24,118003178,235.28,https://www.camara.leg.br/cota-parlamentar/doc...,Copel Distribuição S.A.,04368898000106,235.28,0.0,,1679618.0,0.0
13,Zeca Dirceu,2020.0,2.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,7044385.0,Nota Fiscal,0.0,2020-02-22,122568208,464.77,https://www.camara.leg.br/cota-parlamentar/doc...,Copel Distribuição S.A.,04368898000106,464.77,0.0,,1690671.0,0.0


In [95]:
import plotly.express as px

df_tipo_despesas =df.groupby(['tipoDespesa']).sum('valorLiquido').reset_index()
display(df_tipo_despesas)
fig = px.bar(df_tipo_despesas, x = 'tipoDespesa', y = 'valorLiquido')
fig.show()

,tipoDespesa,ano,mes,codDocumento,codTipoDocumento,valorDocumento,valorLiquido,valorGlosa,codLote,parcela
0,COMBUSTÍVEIS E LUBRIFICANTES.,1446410.0,4605.0,5.079023e+09,1709.0,610180.84,602267.33,7913.51,1.228386e+09,0.0
1,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",181812.0,641.0,6.393504e+08,176.0,1521000.00,1521000.00,0.00,1.548395e+08,0.0
2,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,264623.0,810.0,9.269072e+08,28.0,1149662.18,1143533.17,6129.01,2.236049e+08,0.0
3,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,6605520.0,19417.0,2.314043e+10,2734.0,4197542.16,4172257.04,25008.07,5.583305e+09,0.0
4,PASSAGEM AÉREA - REEMBOLSO,40402.0,204.0,1.425695e+08,33.0,27022.45,27022.45,0.00,3.466138e+07,0.0
5,PASSAGEM AÉREA - RPA,4040.0,2.0,3.448831e+06,0.0,2475.60,2475.60,0.00,0.000000e+00,0.0
6,PASSAGEM AÉREA - SIGEPA,8080.0,47.0,8.528000e+04,0.0,3917.30,3917.30,0.00,0.000000e+00,0.0
7,SERVIÇOS POSTAIS,26263.0,98.0,0.000000e+00,13.0,1044.68,1044.68,0.00,0.000000e+00,0.0
8,TELEFONIA,193941.0,602.0,5.639566e+07,0.0,12456.94,12154.32,150.76,1.355526e+07,0.0


In [98]:
df_despesas_por_deputados = df.groupby(['nome_deputado']).sum('valorLiquido').reset_index()
display(df_despesas_por_deputados)
fig = px.bar(df_despesas_por_deputados, x = 'nome_deputado', y = 'valorLiquido')
fig.show()

,nome_deputado,ano,mes,codDocumento,codTipoDocumento,valorDocumento,valorLiquido,valorGlosa,codLote,parcela
0,AJ Albuquerque,30300.0,96.0,106055234.0,44.0,5779.52,5779.52,0.00,25565039.0,0.0
1,Acácio Favacho,30300.0,66.0,105754591.0,56.0,5836.06,5836.06,0.00,25412729.0,0.0
2,Adolfo Viana,30300.0,105.0,106158291.0,60.0,3865.03,3865.03,0.00,25618311.0,0.0
3,Adriana Ventura,30300.0,57.0,105745810.0,2.0,9626.71,9622.58,4.13,25406418.0,0.0
4,Adriano do Baldy,30300.0,75.0,105962111.0,0.0,6215.41,6202.61,12.80,25522048.0,0.0
...,...,...,...,...,...,...,...,...,...,...
288,Zeca Dirceu,30300.0,84.0,106140517.0,12.0,16814.16,16814.16,0.00,25607288.0,0.0
289,Zé Neto,30300.0,99.0,106114278.0,11.0,74269.88,74266.74,3.14,25594457.0,0.0
290,Zé Silva,30300.0,104.0,106306230.0,16.0,5365.54,5324.70,40.84,25686940.0,0.0
291,Zé Vitor,30300.0,51.0,105627325.0,16.0,2404.47,2404.47,0.00,25353699.0,0.0
